Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [ ]:
import pandas as pd
import sqlite3

%store -r df_bike_station_data
%store -r df_yelp_resultData
%store -r df_fourSquare_resultData

df_bike_station_data = df_bike_station_data
df_fourSquare_resultData = df_fourSquare_resultData
df_yelp_resultData = df_yelp_resultData

first_merged_df = pd.merge(df_bike_station_data, df_fourSquare_resultData, left_index=True, right_on='Key')
second_merged_df = pd.merge(df_bike_station_data, df_yelp_resultData, left_index=True, right_on='Key')

# Add a 'Rating' column to merged_df with null values
first_merged_df['Rating'] = None


# Concatenate the two dataframes based using the 'Key' column
df_final = pd.concat([first_merged_df, second_merged_df], ignore_index=True)
df_final
%store df_final
df_final.head(10)

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# Group by 'Key' and count the occurrences
count_df = df_final.groupby('Key').size().reset_index(name='Count')

# Create a histogram plot
plt.figure(figsize=(10, 6))
sns.barplot(x='Key', y='Count', data=count_df, palette='viridis')
plt.title('Distribution of Count of Stores/Restaurants around each Bike Shop')
plt.xlabel('Bike Shops')
plt.ylabel('Frequency')
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Group by 'Key' and select the first value of 'Number of Bikes' for each group
grouped_df = df_final.groupby('Key').agg({'Number of Bikes': 'first', 'City': 'count'}).reset_index()
grouped_df.columns = ['Key', 'Number of Bikes', 'Count of Stores/Restaurants']

# Create a grouped bar plot
plt.figure(figsize=(12, 6))
sns.barplot(x='Key', y='Count of Stores/Restaurants', data=grouped_df, color='orange', label='Count of Stores/Restaurants')
sns.barplot(x='Key', y='Number of Bikes', data=grouped_df, color='blue', label='Number of Bikes')

plt.title('Comparison of Number of Bikes and Count of Stores/Restaurants around Each Bike Shop')
plt.xlabel('Bike Shops')
plt.ylabel('Count')
plt.legend()
plt.show()

In [ ]:
# Box plot for 'Number of Bikes' to check for outliers
sns.boxplot(x=grouped_df['Number of Bikes'])
plt.show()

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [ ]:
conn = sqlite3.connect('../data/citybikes.db') 
cursor = conn.cursor()

# Create bike_stations table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS bike_stations (
        City TEXT,
        Latitude REAL,
        Longitude REAL,
        Number_of_Bikes INTEGER,
        Key INTEGER,
        PRIMARY KEY (Key)
    )
''')

# Create places table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS places (
        Key INTEGER,
        Name TEXT,
        Category TEXT,
        Distance REAL,
        Rating REAL,
        FOREIGN KEY (Key) REFERENCES bike_stations(Key)
    )
''')

conn.commit()


# Insert data into bike_stations table
df_bike_station_data.to_sql('bike_stations', conn, index=True, if_exists='replace', index_label='Key')


# Insert data into places table
places_df = df_final[['Key', 'Name', 'Category', 'Distance', 'Rating']]
places_df.to_sql('places', conn, index=False, if_exists='replace')

conn.close()

Look at the data before and after the join to validate your data.

In [ ]:
conn = sqlite3.connect('../data/citybikes.db') 
pd.read_sql("""SELECT *
            FROM places
            """, conn
                    )